In [11]:
from gresq.database import sample, preparation_step, dal, Base
from gresq.config import config
import pandas as pd

In [5]:
dal.init_db(config['development'])

In [57]:
Base.metadata.drop_all(bind=dal.engine)
Base.metadata.create_all(bind=dal.engine)

In [58]:
var_map = pd.read_csv('varmap.csv').to_dict()
data = pd.read_csv('data/recipestest.csv').iloc[:-1,:]

In [59]:
col_names = data.columns
with dal.session_scope() as session:
    for i in range(data.shape[0]):
        s = sample()
        for j in range(data.shape[1]):
            value = data.iloc[i,j]
            if pd.isnull(value) == False:
                dbkey = var_map[col_names[j]][0]
                setattr(s,dbkey,value)
        session.add(s)
session.commit()

In [54]:
with dal.session_scope() as session:
    for q in session.query(preparation_step).filter(sample.g_prime_peak_shift<2700):
        print(q.g_prime_peak_shift)

2698.5
2693.0
2695.7
2698.5


In [55]:
data

,NAME,FORMULA,IDENTIFIER,PROPERTY: Average Thickness of Growth (nm),PROPERTY: Standard Deviation of Growth (nm),PROPERTY: Number of Layers,PROPERTY: Growth Coverage (%),PROPERTY: Domain Size (nm),PROPERTY: Geometry,PROPERTY: Silicon Peak Shift (cm$^-1$),PROPERTY: Silicon Peak Amplitude,PROPERTY: Silicon FWHM (cm$^-1$),PROPERTY: D Peak Shift (cm$^-1$),PROPERTY: D Peak Amplitude,PROPERTY: D FWHM (cm$^-1$),PROPERTY: G Peak Shift (cm$^-1$),PROPERTY: G Peak Amplitude,PROPERTY: G FWHM (cm$^-1$),PROPERTY: G' Peak Shift (cm$^-1$),PROPERTY: G' Peak Amplitude,PROPERTY: G' FWHM (cm$^-1$),PROPERTY: No. of Lorenztians under G' Peak,PROPERTY: Sample Surface Area (mm$\^2$),PROPERTY: Thickness ($\mu$m),PROPERTY: Diameter ($\mu$m),PROPERTY: Length (mm),CONDITION ALL: Catalyst,CONDITION ALL: Tube Diameter (mm),CONDITION ALL: Cross Sectional Area (mm$^\2$),CONDITION ALL: Tube Length (mm),CONDITION ALL: Base Pressure (mTorr),PREPARATION STEP NAME,PREPARATION STEP DETAIL: Timestamp A (min),PREPARATION STEP DETAIL: Furnace Temperature A ($^\circ$C),"PREPARATION STEP DETAIL: Furnace Pressure, A (Torr)","PREPARATION STEP DETAIL: Sample Location A, normalized from 0 - 1",PREPARATION STEP DETAIL: Helium Flow Rate A (sccm),PREPARATION STEP DETAIL: Helium Flow Rate A (Torr l/s),PREPARATION STEP DETAIL: Hydrogen Flow Rate A (sccm),PREPARATION STEP DETAIL: Hydrogen Flow Rate A (Torr l/s),PREPARATION STEP DETAIL: Carbon Source,PREPARATION STEP DETAIL: Carbon Source Flow Rate A (sccm),PREPARATION STEP DETAIL: Carbon Source Flow Rate A (Torr l/s),PREPARATION STEP DETAIL: Argon Flow Rate A (sccm),PREPARATION STEP DETAIL: Argon Flow Rate A (Torr l/s),PREPARATION STEP DETAIL: Timestamp B (min),PREPARATION STEP DETAIL: Furnace Temperature B ($^\circ$C),PREPARATION STEP DETAIL: Furnace Pressure B (Torr),"PREPARATION STEP DETAIL: Sample Location B, normalized from 0 - 1",PREPARATION STEP DETAIL: Helium Flow Rate B (sccm),PREPARATION STEP DETAIL: Helium Flow Rate B (Torr l/s),PREPARATION STEP DETAIL: Hydrogen Flow Rate B (sccm),PREPARATION STEP DETAIL: Hydrogen Flow Rate B (Torr l/s),PREPARATION STEP DETAIL: Carbon Source.1,PREPARATION STEP DETAIL: Carbon Source Flow Rate B (sccm),PREPARATION STEP DETAIL: Carbon Source Flow Rate B (Torr l/s),PREPARATION STEP DETAIL: Argon Flow Rate B (sccm),PREPARATION STEP DETAIL: Argon Flow Rate B (Torr l/s),PREPARATION STEP DETAIL: Timestamp C (min),PREPARATION STEP DETAIL: Furnace Temperature C ($^\circ$C),PREPARATION STEP DETAIL: Furnace Pressure C (Torr),"PREPARATION STEP DETAIL: Sample Location C, normalized from 0 - 1",PREPARATION STEP DETAIL: Helium Flow Rate C (sccm),PREPARATION STEP DETAIL: Helium Flow Rate C (Torr l/s),PREPARATION STEP DETAIL: Hydrogen Flow Rate C (sccm),PREPARATION STEP DETAIL: Hydrogen Flow Rate C (Torr l/s),PREPARATION STEP DETAIL: Carbon Source.2,PREPARATION STEP DETAIL: Carbon Source Flow Rate C (sccm),PREPARATION STEP DETAIL: Carbon Source Flow Rate C (Torr l/s),PREPARATION STEP DETAIL: Argon Flow Rate C (sccm),PREPARATION STEP DETAIL: Argon Flow Rate C (Torr l/s),PREPARATION STEP DETAIL: Timestamp D (min),PREPARATION STEP DETAIL: Furnace Temperature D ($^\circ$C),PREPARATION STEP DETAIL: Furnace Pressure D (Torr),"PREPARATION STEP DETAIL: Sample Location D, normalized from 0 - 1",PREPARATION STEP DETAIL: Helium Flow Rate D (sccm),PREPARATION STEP DETAIL: Helium Flow Rate D (Torr l/s),PREPARATION STEP DETAIL: Hydrogen Flow Rate D (sccm),PREPARATION STEP DETAIL: Hydrogen Flow Rate D (Torr l/s),PREPARATION STEP DETAIL: Carbon Source.3,PREPARATION STEP DETAIL: Carbon Source Flow Rate D (sccm),PREPARATION STEP DETAIL: Gas 3 Flow Rate D (Torr l/s),PREPARATION STEP DETAIL: Gas 4 Flow Rate D (sccm),PREPARATION STEP DETAIL: Gas 4 Flow Rate D (Torr l/s),PREPARATION STEP DETAIL: Timestamp E (min),PREPARATION STEP DETAIL: Furnace Temperature E ($^\circ$C),PREPARATION STEP DETAIL: Furnace Pressure E (Torr),"PREPARATION STEP DETAIL: Sample Location E, normalized from 0 - 1",PREPARATION STEP DETAIL: Helium Flow 